In [1]:
clean_up = True
%run StdPackages.ipynb
repo = os.path.join(d['main'],'gams\\AbatementExample')
db_str = os.path.join(repo,'V01_DB.gdx')

In [2]:
os.chdir(d['py'])
import GmsNests, _gamYNests
os.chdir(d['curr'])

### TEST OF GAMS NESTING CLASSES

Set up nesting structure:

In [6]:
t1 = NestingTree.tree('t1', tree = [('s1','Y','L'),('s1','Y','KE'),('s1','KE','x1'),('s1','KE','x2')])
t2 = NestingTree.tree('t2', tree = [('s2','Y','X'),('s2','Y','K')], **{'map': 'NewMapName'})  # this gives the attribute 'map' the name 'NewMapName'
t3 = NestingTree.tree('t3', tree = [('s2','Y','Y1'),('s2','Y','Y2')], io = 'out')

*Add other variables to namespaces etc:*

In [7]:
t = NestingTree.AggTree(name='agg', trees ={ti.name: ti for ti in (t1,t2,t3)})
t.run()
t.ns['t'] = 'year'
t.ns.update({k:k for k in ('qD','qS','pS','pD','mu','sigma','txE')})

In [8]:
ns = t.ns
ns_i = t1.ns
db = t.db
name = t1.name

In [9]:
ns_i

{'n': 'n',
 'nn': 'nn',
 'nnn': 'nnn',
 's': 's',
 'map': 'map_t1',
 'knot': 'knot_t1',
 'branch': 'branch_t1',
 'input': 'input_t1',
 'output': 'output_t1',
 'knot_o': 'knot_o_t1',
 'knot_no': 'knot_no_t1',
 'branch2o': 'branch2o_t1',
 'branch2no': 'branch2no_t1'}

*Add time:*

In [10]:
db['year'] = pd.Index(range(2),name='year')
db['txE'] = pd.Index([0],name='year')

*Add variables:*

In [11]:
db['qD'] = pd.Series(1, index = pd.MultiIndex.from_product([db.get('year'),db.get('s'),db.get('n')]),name='qD')
db['qS'] = pd.Series(1, index = pd.MultiIndex.from_product([db.get('year'),db.get('s'),db.get('n')]),name='qS')
db['pD'] = pd.Series(1, index = pd.MultiIndex.from_product([db.get('year'),db.get('s'),db.get('n')]),name='pD')
db['pS'] = pd.Series(1, index = pd.MultiIndex.from_product([db.get('year'),db.get('s'),db.get('n')]),name='pS')
db['sigma'] = pd.Series(1, index = pd.MultiIndex.from_product([db.get('s'),db.get('n')]), name = 'sigma')
db['mu'] = pd.Series(1, index = pd.MultiIndex.from_product([db.get('s'),db.get('n'),db.get('n').rename('nn')]),name='mu')

#### CES test:

In [20]:
t.get('branch2o',local='t1')

MultiIndex([('s1',  'L'),
            ('s1', 'KE')],
           names=['s', 'n'])

In [18]:
t.get('map',local='t1')

MultiIndex([('s1',  'Y',  'L'),
            ('s1',  'Y', 'KE'),
            ('s1', 'KE', 'x1'),
            ('s1', 'KE', 'x2')],
           names=['s', 'n', 'nn'])

In [12]:
a = GmsNests.CES(ns,ns_i,db,name)
print(a.block)

$BLOCK B_t1
	E_zp_out_t1[year,s,n]$(knot_o_t1[s,n] and txE[year])..	pS[year,s,n]*qS[year,s,n] =E= sum(nn$(map_t1[s,nn,n]),qD[year,s,nn]*pD[year,s,nn]);
	E_zp_nout_t1[year,s,n]$(knot_no_t1[s,n] and txE[year])..	pD[year,s,n]*qD[year,s,n] =E= sum(nn$(map_t1[s,nn,n]),qD[year,s,nn]*pD[year,s,nn]);
	E_demand_out_t1[year,s,n]$(branch2o_t1[s,n] and txE[year])..	qD[year,s,n] =E= sum(nn$(map_t1[s,n,nn]), mu[s,n,nn] * (pS[year,s,nn]/pD[year,s,n])**(sigma[s,nn]) * qS[year,s,nn]);
	E_demand_nout_t1[year,s,n]$(branch2no_t1[s,n] and txE[year])..	qD[year,s,n] =E= sum(nn$(map_t1[s,n,nn]), mu[s,n,nn] * (pD[year,s,nn]/pD[year,s,n])**(sigma[s,nn]) * qD[year,s,nn]);
$ENDBLOCK


#### CES_norm test:

In [13]:
b = GmsNests.CES_norm(ns,ns_i,db,name)
print(b.block)

$BLOCK B_t1
	E_zp_out_t1[year,s,n]$(knot_o_t1[s,n] and txE[year])..	pS[year,s,n]*qS[year,s,n] =E= sum(nn$(map_t1[s,nn,n]),qD[year,s,nn]*pD[year,s,nn]);
	E_zp_nout_t1[year,s,n]$(knot_no_t1[s,n] and txE[year])..	pD[year,s,n]*qD[year,s,n] =E= sum(nn$(map_t1[s,nn,n]),qD[year,s,nn]*pD[year,s,nn]);
	E_demand_out_t1[year,s,n]$(branch2o_t1[s,n] and txE[year])..	qD[year,s,n] =E= sum(nn$(map_t1[s,n,nn]), mu[s,n,nn] * (pS[year,s,nn]/pD[year,s,n])**(sigma[s,nn]) * qS[year,s,nn] / sum(nnn$(map_t1[s,nnn,nn]), mu[s,nnn,nn] * (pS[year,s,nn]/pD[year,s,nnn])**(sigma[s,nn])));
	E_demand_out_t1[year,s,n]$(branch2o_t1[s,n] and txE[year])..	qD[year,s,n] =E= sum(nn$(map_t1[s,n,nn]), mu[s,n,nn] * (pD[year,s,nn]/pD[year,s,n])**(sigma[s,nn]) * qD[year,s,nn] / sum(nnn$(map_t1[s,nnn,nn]), mu[s,nnn,nn] * (pD[year,s,nn]/pD[year,s,nnn])**(sigma[s,nn])));
$ENDBLOCK


#### MNL test:

In [14]:
c = GmsNests.MNL(ns,ns_i,db,name)
print(c.block)

$BLOCK B_t1
	E_zp_out_t1[year,s,n]$(knot_o_t1[s,n] and txE[year])..	pS[year,s,n]*qS[year,s,n] =E= sum(nn$(map_t1[s,nn,n]),qD[year,s,nn]*pD[year,s,nn]);
	E_zp_nout_t1[year,s,n]$(knot_no_t1[s,n] and txE[year])..	pD[year,s,n]*qD[year,s,n] =E= sum(nn$(map_t1[s,nn,n]),qD[year,s,nn]*pD[year,s,nn]);
	E_demand_out_t1[year,s,n]$(branch2o_t1[s,n] and txE[year])..	qD[year,s,n] =E= sum(nn$(map_t1[s,n,nn]), mu[s,n,nn] * exp((pS[year,s,nn]-pD[year,s,n])*(sigma[s,nn])) * qS[year,s,nn] / sum(nnn$(map_t1[s,nnn,nn]), mu[s,nnn,nn] * exp((pS[year,s,nn]-pD[year,s,nnn])*(sigma[s,nn]))));
	E_demand_out_t1[year,s,n]$(branch2o_t1[s,n] and txE[year])..	qD[year,s,n] =E= sum(nn$(map_t1[s,n,nn]), mu[s,n,nn] * exp((pD[year,s,nn]-pD[year,s,n])*(sigma[s,nn])) * qD[year,s,nn] / sum(nnn$(map_t1[s,nnn,nn]), mu[s,nnn,nn] * exp((pD[year,s,nn]-pD[year,s,nnn])*(sigma[s,nn]))));
$ENDBLOCK


#### CET test:

In [21]:
ns = t.ns
ns_i = t3.ns
db = t.db
name = t3.name

In [22]:
d = GmsNests.CET(ns,ns_i,db,name)
print(d.block)

$BLOCK B_t3
	E_zp_t3[year,s,n]$(knot_t3[s,n] and txE[year])..	pD[year,s,n]*qD[year,s,n] =E= sum(nn$(map_t3[s,nn,n] and branch_o_t3[s,nn]), qS[year,s,nn]*pS[year,s,nn])+sum(nn$(map_t3[s,nn,n] and branch_no_t3[s,nn]), qD[year,s,nn]*pD[year,s,nn]);
	E_demand_out_t3[year,s,n]$(branch_o_t3[s,n] and txE[year])..	qS[year,s,n] =E= sum(nn$(map_t3[s,n,nn]), mu[s,n,nn] * (pS[year,s,n]/pD[year,s,nn])**(sigma[s,nn]) * qD[year,s,nn]);
	E_demand_nout_t3[year,s,n]$(branch_no_t3[s,n] and txE[year])..	qD[year,s,n] =E= sum(nn$(map_t3[s,n,nn]), mu[s,n,nn] * (pD[year,s,n]/pD[year,s,nn])**(sigma[s,nn]) * qD[year,s,nn]);
$ENDBLOCK


#### CET_norm test:

In [23]:
e = GmsNests.CET_norm(ns,ns_i,db,name)
print(e.block)

$BLOCK B_t3
	E_zp_t3[year,s,n]$(knot_t3[s,n] and txE[year])..	pD[year,s,n]*qD[year,s,n] =E= sum(nn$(map_t3[s,nn,n] and branch_o_t3[s,nn]), qS[year,s,nn]*pS[year,s,nn])+sum(nn$(map_t3[s,nn,n] and branch_no_t3[s,nn]), qD[year,s,nn]*pD[year,s,nn]);
	E_demand_out_t3[year,s,n]$(branch_o_t3[s,n] and txE[year])..	qS[year,s,n] =E= sum(nn$(map_t3[s,n,nn]), mu[s,n,nn] * (pS[year,s,n]/pD[year,s,nn])**(sigma[s,nn]) * qD[year,s,nn]) / (sum(nnn$(map_t3[s,nnn,nn] and branch_o_t3[s,nnn]), mu[s,nnn,nn]*(pS[year,s,nnn]/pD[year,s,nn])**(sigma[s,nn])+sum(nnn and branch_no_t3[s,nnn]), mu[s,nnn,nn]*(pD[year,s,nnn]/pD[year,s,nn])**(sigma[s,nn])));
	E_demand_nout_t3[year,s,n]$(branch_no_t3[s,n] and txE[year])..	qD[year,s,n] =E= sum(nn$(map_t3[s,n,nn]), mu[s,n,nn] * (pD[year,s,n]/pD[year,s,nn])**(sigma[s,nn]) * qD[year,s,nn]) / (sum(nnn$(map_t3[s,nnn,nn] and branch_o_t3[s,nnn]), mu[s,nnn,nn]*(pS[year,s,nnn]/pD[year,s,nn])**(sigma[s,nn])+sum(nnn and branch_no_t3[s,nnn]), mu[s,nnn,nn]*(pD[year,s,nnn]/pD[year,s,n

#### MNL_out test:

In [15]:
f = GmsNests.CET_norm(ns,ns_i,db,name)
print(f.block)

$BLOCK B_t3
	E_zp_t3[year,s,n]$(knot_t3[s,n] and txE[year])..	pD[year,s,n]*qD[year,s,n] =E= sum(nn$(map_t3[s,nn,n] and branch_o_t3[s,nn]), qS[year,s,nn]*pS[year,s,nn])+sum(nn$(map_t3[s,nn,n] and branch_no_t3[s,nn]), qD[year,s,nn]*pD[year,s,nn]);
	E_demand_out_t3[year,s,n]$(branch_o_t3[s,n] and txE[year])..	qS[year,s,n] =E= sum(nn$(map_t3[s,n,nn]), mu[s,n,nn] * (pS[year,s,n]/pD[year,s,nn])**(sigma[s,nn]) * qD[year,s,nn]) / (sum(nnn$(map_t3[s,nnn,nn] and branch_o_t3[s,nnn]), mu[s,nnn,nn]*(pS[year,s,nnn]/pD[year,s,nn])**(sigma[s,nn])+sum(nnn and branch_no_t3[s,nnn]), mu[s,nnn,nn]*(pD[year,s,nnn]/pD[year,s,nn])**(sigma[s,nn])));
	E_demand_nout_t3[year,s,n]$(branch_no_t3[s,n] and txE[year])..	qD[year,s,n] =E= sum(nn$(map_t3[s,n,nn]), mu[s,n,nn] * (pD[year,s,n]/pD[year,s,nn])**(sigma[s,nn]) * qD[year,s,nn]) / (sum(nnn$(map_t3[s,nnn,nn] and branch_o_t3[s,nnn]), mu[s,nnn,nn]*(pS[year,s,nnn]/pD[year,s,nn])**(sigma[s,nn])+sum(nnn and branch_no_t3[s,nnn]), mu[s,nnn,nn]*(pD[year,s,nnn]/pD[year,s,n